# Smoke Sensor Anomaly Detection

## Import Necessary Libraries

In [1]:
## Import necessary libraries
import pandas as pd
import numpy as np
import random 
import pickle
from urllib.parse import quote
from sklearn.preprocessing import MinMaxScaler

## Import necessary API
import sys
sys.path.append('../../../../')
from api.v2.util.data_load import data_load
from api.v2.model.ResNet1d import ResNet1D, ResidualBlock

## Import libraries for the model
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## Set path for saving model training results 
import os
os.makedirs('./result', exist_ok=True)

## Set Cuda for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Set random seed
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# Set seed
seed_val = 77
set_seed(seed_val)

cuda


## Base Parameter Setting
* Set parameters based on the information identified during EDA (Exploratory Data Analysis).

In [2]:
# Set Smoke Sensor Data Parameter

# Set MachBase Neo URL address
URL = 'http://127.0.0.1:5654'
# Set Tag Table Name
table = 'smoke'
# Select Tag Name -> Can Check Tag Names Using command 'show_column(URL, table)'
# Set Austria Tag Name 
tags = ['CNT', 'Fire Alarm', 'Humidity[%]', 'PM1.0', 'Pressure[hPa]', 'Raw Ethanol', 'Raw H2', 'TVOC[ppb]', 'Temperature[C]', 'eCO2[ppm]']
# Wrap each item in the list with single quotes and separate with commas
tags_ = ",".join(f"'{tag}'" for tag in tags)
# Set Tag Name
name = quote(tags_, safe=":/")
# Set resample Option -> D(day), H(hour), T(minute), S(second)
resample_freq = None
# Set Start time
start_time = '2025-01-16 00:00:00'
# Set End time 
end_time = '2025-01-16 17:23:49'
# Set TimeFormat - > 'default' or quote('2006-01-02 15:04:05.000000')(Divided down to the nanosecond)
timeformat = 'default'

## Smoke Sensor Data Load

In [3]:
# Smoke Sensor Data Load
df = data_load(URL, table, name, start_time, end_time, timeformat, resample_freq)

# Move the 'Fire Alarm' column to the last position
df = df.reindex(columns=[col for col in df.columns if col != 'Fire Alarm'] + ['Fire Alarm'])

# Convert the 'Fire Alarm' column to integer type
df['Fire Alarm'] = df['Fire Alarm'].astype(int)

In [4]:
df

NAME,CNT,Humidity[%],PM1.0,Pressure[hPa],Raw Ethanol,Raw H2,TVOC[ppb],Temperature[C],eCO2[ppm],Fire Alarm
TIME,,,,,,,,,,
2025-01-16 00:00:00,0.0,57.36,0.00,939.735,18520.0,12306.0,0.0,20.000,400.0,0
2025-01-16 00:00:01,1.0,56.67,0.00,939.744,18651.0,12345.0,0.0,20.015,400.0,0
2025-01-16 00:00:02,2.0,55.96,0.00,939.738,18764.0,12374.0,0.0,20.029,400.0,0
2025-01-16 00:00:03,3.0,55.28,0.00,939.736,18849.0,12390.0,0.0,20.044,400.0,0
2025-01-16 00:00:04,4.0,54.69,0.00,939.744,18921.0,12403.0,0.0,20.059,400.0,0
...,...,...,...,...,...,...,...,...,...,...
2025-01-16 17:23:45,5739.0,15.79,0.63,936.670,20569.0,13723.0,625.0,18.438,400.0,0
2025-01-16 17:23:46,5740.0,15.87,0.61,936.678,20588.0,13731.0,612.0,18.653,400.0,0
2025-01-16 17:23:47,5741.0,15.84,0.57,936.687,20582.0,13725.0,627.0,18.867,400.0,0


In [5]:
# Split the data into train, test sets
train = df[df.index.hour < 10]
test = df[df.index.hour >= 10]

## Data Preprocessing
* 1 Min-Max Scaling

### 1. Applying Min-Max Scaling

In [6]:
# Set up Scalers
scaler = MinMaxScaler()

# Apply Scalers
train_ = scaler.fit_transform(train.iloc[:,:-1].values)
test_ = scaler.transform(test.iloc[:,:-1].values)

# Set Each DataFrames
train_scaled = pd.DataFrame(train_, columns=train.columns[:-1])
train_scaled['Fire Alarm'] = train['Fire Alarm'].values
  
test_scaled = pd.DataFrame(test_, columns=test.columns[:-1])
test_scaled['Fire Alarm'] = test['Fire Alarm'].values

# Save Scaler
with open('./result/resnet1d_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## Dataset & Loader Setup

In [7]:
class Smoke_Dataset(Dataset):

    def __init__(self, df):
        self.freq_data = df.iloc[:,:-1]
        self.label = df.iloc[:,-1:].values

    def __len__(self):
        return len(self.freq_data)

    def __getitem__(self, index):

        input_time_data = self.freq_data.iloc[index,:]
        input_time_data = torch.Tensor(input_time_data).expand(1, input_time_data.shape[0])
        label = self.label[index]

        return input_time_data, label

In [8]:
# Set up datasets  
train_ = Smoke_Dataset(train_scaled)
test_ = Smoke_Dataset(test_scaled)

# Set up data loaders
train_dataloader = DataLoader(train_, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_, batch_size=32, shuffle=False)

In [9]:
# Verify DataLoader application and check the shape of the input data
print(list(train_dataloader)[0][0].shape)

torch.Size([32, 1, 9])


## Model Configuration
* Using ResNet1d model.

In [10]:
# Model configuration parameters
 
# Set ResidualBlock
block = ResidualBlock
# Set the number of ResidualBlocks to use per layer
layers = [2,2,2,2]
# Set the number of classification categories
num_classes = 2
 
# Learning rate
lr = 0.01
 
# Model configuration
model = ResNet1D(block, layers, num_classes).to(device)
 
# Configure loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
 
# Check the model architecture
print(model)

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), paddi

## Model Training

* Save the model with the Best F1 Score based on the validation data during training.

In [11]:
# Initialize training loss
train_loss = []
# Initialize training accuracy
train_acc = []
# Initialize total step
total_step = len(train_dataloader)
# Set number of epochs
epoch_in = trange(5, desc='training')
# Initialize best F1 Score value
best_f1= 0

# Start model training
for epoch in epoch_in:

    model.train()
    running_loss = 0.0
    correct = 0
    total=0
    preds_ = []
    targets_ = []
    
    # Initialize loss
    train_loss = []
    
    for batch_idx, (data, target) in enumerate(train_dataloader):
        
        data = data.to(device).float()
        target = target.to(device).long().squeeze()
        
        optimizer.zero_grad()
        
        # Input to the model
        outputs = model(data)
    
        # Calculate loss
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # Set label predictions 
        _,pred = torch.max(outputs, dim=1)
        target_ = target.view_as(pred)
        correct += torch.sum(pred==target).item()
        total += target.size(0)
        
        preds_.append(pred)
        targets_.append(target_)
            
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')

    # Combine predictions and labels collected from all batches
    preds_ = torch.cat(preds_).detach().cpu().numpy()
    targets_ = torch.cat(targets_).detach().cpu().numpy()
    
    f1score = f1_score(targets_, preds_,  average='macro')
    if best_f1 < f1score:
        best_f1 = f1score
        # Save the best model 
        with open("./result/Smoke_Sensor_Full.txt", "a") as text_file:
            print('epoch=====',epoch, file=text_file)
            print(classification_report(targets_, preds_, digits=4), file=text_file)
        print('model save')
        torch.save(model, f'./result/Smoke_Sensor_Full.pt') 
    epoch_in.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")

training:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



train loss: 0.04547461824055022, train acc: 98.5944
model save

train loss: 0.014271776547288634, train acc: 99.5222
model save

train loss: 0.008839443033903485, train acc: 99.7333
model save

train loss: 0.011336452931209995, train acc: 99.6917

train loss: 0.009640899327687495, train acc: 99.7000


## Model Testing

In [12]:
# Load the best model
model_ = torch.load(f'./result/Smoke_Sensor_Full.pt')

In [13]:
# Model testing
preds_test = []
target_test = []
with torch.no_grad():
    model_.eval()
    for batch_idx, (data, target) in enumerate(test_dataloader):
        data = data.to(device).float()
        target = target.to(device).long().squeeze()
        
        outputs_t = model_(data)
        
        _,pred_t = torch.max(outputs_t, dim=1)
        targets_t = target.view_as(pred_t).to(device)

        preds_test.append(pred_t)
        target_test.append(targets_t)
        
    # Combine predictions and labels collected from all batches
    preds_test = torch.cat(preds_test).detach().cpu().numpy()
    target_test = torch.cat(target_test).detach().cpu().numpy()

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


## Model Performance Evaluation

In [14]:
print(classification_report(target_test, preds_test))

              precision    recall  f1-score   support

           0       0.90      0.84      0.87     11517
           1       0.88      0.93      0.90     15113

    accuracy                           0.89     26630
   macro avg       0.89      0.88      0.89     26630
weighted avg       0.89      0.89      0.89     26630

